In [1]:
import ee
ee.Initialize(project='pyregence-ee')
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
#individual fire export uses geetools
import geetools
%load_ext autoreload
%autoreload 2

### Run Burn Severity for a particular year

#### Production run with NIFC

In [2]:
# For running a year of fires from NIFC for burn severity
#  Based the corrections in the simulation code, but no fire filter dates, 
#   and replaced 'simulation' date with run date (the current date)
# REQUIRES USER INPUT/SELECTION, see starred section below. 

# Last edited 2023-09-27 by dnekorchuk

#***
# USER: 
# 1. Set data import, align year and import dataset
# 2. Change data_origin filename tag and asset out folder as needed
# 3. Scroll down and set export flags
# 4. Select window type
#***

#### IMPORT DATA ******************************************

## NIFC 
yr = '2023'
#fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/nifc/nifc_fires_2022_gte100ac_20230406')
fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/nifc/nifc_fires_2023_gte5000ac_20231020')
#relevant field names: Acres, Discovery, GlobalID, Name
name_field = 'Name'
id_field = 'GlobalID'
size_field = 'Acres'

#For testing subsets of fires
# id_list = ['dd114282-a54d-414b-bdb1-4e180fc1ea7d', '221b33e5-594c-4f68-9131-9d412e057dfd']
# fires_known = fires_raw.filter(ee.Filter.inList(id_field,id_list))
# #plus some other fires
# fires_add = fires_raw.limit(22, 'Discovery', False) 
# fires = fires_known.merge(fires_add)

#For run with all fires
fires = fires_raw

data_origin = 'nifc2023test1'
asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing' #'projects/pyregence-ee/assets/conus/nifc'


## USER SETTINGS **********************************

export_flag = True #severity classes, composite/mosiac conus
per_fire_export = False #indiv fire results for dNBR and severity. #trying fitoprincipe batch export, failing
csv_export_flag = True #csv log/diagnostic dates and info
#Variants of recent fire windows
# 'fixed' - fixed 90 days post fire, but won't go past sim date
# 'expanding' - variable days expanding from fire to current/sim date
# 'sliding' - fixed 90 day window, sliding along to most recent 90 days to current/sim date.  
recent_type = 'sliding' 

##### SET RUN DATE #####

#Today
run_date = datetime.datetime.now()
run_date_ee = ee.Date(run_date)
run_date_formatted = str(run_date)[0:10]

# Map over the fires, adding the run date 
#  note: must use subfunction set_windows_sim() in bs calc function
#  Also add id field for labeling bands in exported intermediate products
def set_run_date(feat: ee.Feature):
    fire_date = ee.Date(feat.getString('Discovery'))
    #calculate number days from Discovery to run date (negative is before run date)
    days_from_run = fire_date.difference(run_date_ee,'day')
    #Create a name for the individual fire export. Not currently used until indiv export fixed.
    #concatenate name pieces. Note: cannot start with number, if doing band-method. 
    fire_id_prefix = ee.String('Fire_')
    fire_name_raw = ee.String(feat.get(name_field))
    # cannot have () and probably other special characters in band names, also removes all whitespace
    fire_name = fire_name_raw.replace('[^a-zA-Z0-9]', "", 'g')
    sep = ee.String('_')
    fire_event_id = ee.String(feat.get(id_field))
    fire_id = fire_id_prefix.cat(fire_name).cat(sep).cat(fire_event_id)
    return feat.set('run_date',run_date_formatted, 
                    'days_from_run',days_from_run,
                    'fire_id',fire_id,
                    'recent_type',recent_type)
fires = fires.map(set_run_date)


##### METADATA COLLECTION 1 #####
# Metadata collection and print
fires_count = fires.size().getInfo()
print(f'Total Fires in FeatureCollection: {fires_count}')

metadata = {
    'run_date':run_date.strftime('%Y%m%d'),
    'fire_count':fires_count,
    'code_origin':'BS_Mapper_FF3',
    'recent_type':recent_type
}
#print(metadata)


###### GET FIRE WINDOW DIAGNOSTICS ######

bs_windows = ee.FeatureCollection(fires.map(af.bs_get_windows_ff3))

desc_windows = f'bs_{yr}_{data_origin}_recent{recent_type}_windows_{run_date.strftime("%Y%m%d")}'
#wanted properties. Event_ID only in MTBS, will be empty in NIFC runs
wanted_cols = [
    id_field, name_field, size_field, #'Event_ID',
    'run_date', 'days_from_run', 'Discovery', 
    'mode', 'recent_type', 'pre_start', 'pre_end', 'post_start', 'post_end', 'double_check']
task_csv = ee.batch.Export.table.toDrive(
    collection=bs_windows,
    description=desc_windows,
    selectors=wanted_cols,
    folder='Fire',
    fileFormat='CSV')

if csv_export_flag == True:
    print(f'export task started: fire window diagnostics {desc_windows} to Google drive of user.')
    task_csv.start()


##### RUN BURN SEVERITY #####

# Returns RdNBR & BS for each fire ee.Feature
bs_coll_combined = ee.FeatureCollection(fires).map(af.bs_calc_ff3)

# Separate out RdNBR and Miller Threshold severity classes
#bs_rdnbr = ee.ImageCollection(bs_coll_combined).select('RdNBR')
bs_coll = ee.ImageCollection(bs_coll_combined).select('MillersThresholds')


##### Export intermediate: individual features #####

if per_fire_export == True:

    #print('INDIVIDUAL FIRES NOT RUN: Need to write export.')

    desc_indiv = f'bs_{yr}_{data_origin}_recent{recent_type}_perfire_{run_date.strftime("%Y%m%d")}'
    
    # TODO: batch export isn't picking up the path correctly??
    # Parameter "name" value "projects/pyregence-ee/assets" does not match the pattern "^projects/[^/]+/assets/.*$"
    
    path_coll = asset_folder+"/"+desc_indiv #f'{asset_folder}/{desc_indiv}'
    print(f'individual fire image collection: {path_coll}') 
    
    #hard code test
    #asset_path = 'projects/pyregence-ee/assets/bs_simrun2020_20200930_mtbs4_recentfixed_perfire_20230925'

    #use geetools to batch export to an image collection
    geetools.batch.Export.imagecollection.toAsset(
        collection=bs_coll_combined, 
        assetPath=path_coll, #asset_path,  
        namePattern='{fire_id}', 
        scale=30,
        dataType="float", 
        region=None, 
        datePattern=None,
        extra=None, 
        verbose=False
    )

else:
    print('Skipping individual fire export')


###### METADATA COLLECTION 3 ######

#fire_calc_count = ee.ImageCollection(bs_coll_combined).size() # always an image, but some are no data
# Instead add up flag (0/1) counts in 'post_fire_calc'
fire_calc_count = ee.ImageCollection(bs_coll_combined).aggregate_sum('post_fire_calc')
metadata['fire_count_results'] = fire_calc_count

##### COMPOSITE BURN SEVERITY & FINAL EXPORT #####

# composite that ee.ImageCollection with a max() reducer, add metadata
bs_composite = bs_coll.max().rename('SEVERITY').set('Year',int(yr)).set(metadata)

# To Asset
desc = f'bs_{yr}_{data_origin}_recent{recent_type}_{run_date.strftime("%Y%m%d")}'

utils.exportImgtoAsset(bs_composite, 
                    desc=desc,
                    region=None,
                    asset_folder=asset_folder, 
                    export_type='conus',
                    export=export_flag)



Total Fires in FeatureCollection: 90
export task started: fire window diagnostics bs_2022_nifc2023test1_recentsliding_windows_20231020 to Google drive of user.
Skipping individual fire export
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/bs_2022_nifc2023test1_recentsliding_20231020
